In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,root_mean_squared_error

In [ ]:
# Load the dataset
df = pd.read_csv('expanded_pc_components.csv')

In [ ]:
label_encoders = {}
categorical_cols = ['case_Model', 'cpu_Model', 'internal-hard-drive_Model',
                   'memory_Model', 'motherboard_Model', 'power-supply_Model',
                   'video-card_Model']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    # Initialize label encoders for categorical features

In [ ]:
# Separate features and target
X = df.drop('Total Price', axis=1)
y = df['Total Price']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def build_model(input_shape):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])


    learning_rate = 0.001
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Build and train the model
model = build_model(X_train_scaled.shape[1])
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_test_scaled, y_test),
                    epochs=3000, batch_size=32, verbose=2)

Epoch 1/3000


c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 - 2s - 8ms/step - loss: 52409500.0000 - mae: 4670.6333 - val_loss: 1857368.2500 - val_mae: 1081.9645
Epoch 2/3000
250/250 - 0s - 1ms/step - loss: 1604110.5000 - mae: 1011.3575 - val_loss: 1469393.2500 - val_mae: 966.5250
Epoch 3/3000
250/250 - 0s - 1ms/step - loss: 1280949.6250 - mae: 902.9905 - val_loss: 1164556.0000 - val_mae: 857.7271
Epoch 4/3000
250/250 - 0s - 1ms/step - loss: 1013532.6250 - mae: 799.8098 - val_loss: 916907.0625 - val_mae: 764.7186
Epoch 5/3000
250/250 - 0s - 1ms/step - loss: 792912.8125 - mae: 710.5421 - val_loss: 722043.1875 - val_mae: 676.6982
Epoch 6/3000
250/250 - 0s - 1ms/step - loss: 599107.3750 - mae: 614.7424 - val_loss: 520501.5938 - val_mae: 574.3380
Epoch 7/3000
250/250 - 0s - 1ms/step - loss: 431180.4062 - mae: 521.2499 - val_loss: 372200.7500 - val_mae: 481.9685
Epoch 8/3000
250/250 - 0s - 1ms/step - loss: 283704.9062 - mae: 419.5596 - val_loss: 223440.8594 - val_mae: 370.9636
Epoch 9/3000
250/250 - 0s - 1ms/step - loss: 167935.1094 - mae: 32

In [ ]:
with open("pc_price_model.pkl", "wb") as f:
    pickle.dump(model, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)